In [1]:
import cnvpytor
import subprocess

In [2]:
filename = "test_with_del_400_3_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip()) + 1
line_count

1050026

In [3]:
command = f"wgsim -d 300 -N {line_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1699991817
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 52501171


b'chr22\t1290\tA\tC\t-\nchr22\t2236\tA\t-\t-\nchr22\t2583\tTC\t-\t-\nchr22\t3192\tC\tS\t+\nchr22\t4626\t-\tT\t+\nchr22\t4680\tT\tW\t+\nchr22\t5786\tA\tW\t+\nchr22\t6356\tT\tK\t+\nchr22\t6583\tC\tS\t+\nchr22\t6678\tT\tY\t+\nchr22\t9019\tG\tK\t+\nchr22\t9323\t-\tG\t+\nchr22\t10115\tC\tS\t+\nchr22\t10199\tC\tY\t+\nchr22\t10644\tT\tA\t-\nchr22\t10714\tC\tY\t+\nchr22\t11431\tA\t-\t-\nchr22\t12153\tG\tK\t+\nchr22\t12379\tC\tT\t-\nchr22\t14621\t-\tG\t+\nchr22\t14879\tT\tA\t-\nchr22\t15081\t-\tC\t-\nchr22\t16627\tT\tY\t+\nchr22\t19833\tG\tR\t+\nchr22\t19935\tT\tK\t+\nchr22\t20410\tC\tS\t+\nchr22\t20851\tC\tA\t-\nchr22\t22031\t-\tC\t+\nchr22\t22149\tT\tK\t+\nchr22\t22847\tT\tA\t-\nchr22\t24478\tG\tS\t+\nchr22\t24593\tC\tM\t+\nchr22\t25128\tT\tG\t-\nchr22\t25189\tA\tG\t-\nchr22\t29157\tG\tT\t-\nchr22\t29330\tA\tW\t+\nchr22\t30443\tG\tK\t+\nchr22\t30763\tG\tA\t-\nchr22\t34430\tA\tC\t-\nchr22\t35161\tA\tG\t-\nchr22\t36045\tG\tA\t-\nchr22\t39742\tG\tT\t-\nchr22\t40047\tT\tA\t-\nchr22\t40827\tC\tT\t

In [4]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [5]:
# sam to bam file conversion
!samtools view -bS biasOut.sam > biasOut.bam
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam


[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [6]:
print(cnvpytor.__version__)
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)
app.rd(["biasOut_sorted.bam"])

1.3.1


In [7]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000], print_calls=True)

In [8]:
for call in calls[100]:
    print(call)

In [9]:
for call in calls[1000]:
    print(call)

In [10]:
for call in calls[10000]:
    print(call[:5])

In [11]:
for call in calls[100000]:
    print(call)

In [12]:
for call in calls[1000000]:
    print(call)

In [13]:
del app